In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/interbank20/sample_submission.csv
/kaggle/input/interbank20/y_train.csv
/kaggle/input/interbank20/sunat_test.csv
/kaggle/input/interbank20/rcc_test.csv
/kaggle/input/interbank20/se_test.csv
/kaggle/input/interbank20/sunat_train.csv
/kaggle/input/interbank20/se_train.csv
/kaggle/input/interbank20/censo_test.csv
/kaggle/input/interbank20/productos.csv
/kaggle/input/interbank20/censo_train.csv
/kaggle/input/interbank20/rcc_train.csv


In [2]:
rcc_train = pd.read_csv('/kaggle/input/interbank20/rcc_train.csv')
rcc_test = pd.read_csv('/kaggle/input/interbank20/rcc_test.csv')
y_train = pd.read_csv('/kaggle/input/interbank20/y_train.csv', index_col='key_value')

In [3]:
bins = [-1, 0, 10, 20, 30, 60, 90, 180, 360, 720, float("inf")]
rcc_train["condicion"] = pd.cut(rcc_train.condicion, bins, labels = range(0,10)).astype('int32')
rcc_test["condicion"] = pd.cut(rcc_test.condicion, bins, labels = range(0,10)).astype('int32')
rcc_test['cod_instit_financiera'].fillna(rcc_test['cod_instit_financiera'].value_counts().index[0], inplace=True)
rcc_test['PRODUCTO'].fillna(rcc_test['PRODUCTO'].value_counts().index[0], inplace=True)

In [4]:
dict_ = dict(zip(list(rcc_train.dtypes[rcc_train.dtypes == int].index), ['int32']*len(rcc_train.dtypes[rcc_train.dtypes == int].index)))
rcc_train = rcc_train.astype(dict_)
rcc_test = rcc_test.astype(dict_)

In [5]:
import statistics as stat
import numpy as np
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=np.nan
    return moda

In [6]:
from tqdm.notebook import tqdm 
import gc
list_rcc_train_agg = []

moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_rcc = {'condicion':['max'],
           'saldo':['max'],
           'cod_instit_financiera':['nunique',moda],
           'PRODUCTO':['nunique',moda],
           'COD_CLASIFICACION_DEUDOR':['nunique','max','min','sum',moda]
          }

In [7]:
for n,i in enumerate(tqdm(sorted(set(rcc_train.codmes),reverse=True))):
#     if i in [201802,201712,201709,201703]:
    if n in [0,1,2,3,4,5,11]:
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldo_ult{n+1}meses' for c in rcc_train_agg.columns]
            list_rcc_train_agg.append(rcc_train_agg)
            gc.collect()
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).condicion.max().unstack(level = 1, fill_value=0)
            rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_condicion_ult{n+1}meses' for c in rcc_train_agg.columns]
            list_rcc_train_agg.append(rcc_train_agg)
            gc.collect()

        print(f'haciendo agg ')
        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby('key_value').agg(agg_rcc)
        rcc_train_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()
        
rcc_train_ = pd.concat(list_rcc_train_agg, axis=1)
del rcc_train, list_rcc_train_agg

haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haci

In [8]:
from tqdm.notebook import tqdm 
list_rcc_test_agg = []
for n,i in enumerate(tqdm(sorted(set(rcc_test.codmes),reverse=True))):
#     if i in [201902,201812,201809,201803]:
    if n in [0,1,2,3,4,5,11]:
        for c in ['tipo_credito','cod_instit_financiera','PRODUCTO','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldo_ult{n+1}meses' for c in rcc_test_agg.columns]
            list_rcc_test_agg.append(rcc_test_agg)
            gc.collect()
        for c in ['tipo_credito','cod_instit_financiera','PRODUCTO','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).condicion.max().unstack(level = 1, fill_value=0)
            rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_condicion_ult{n+1}meses' for c in rcc_test_agg.columns]
            list_rcc_test_agg.append(rcc_test_agg)
            gc.collect()
        print(f'haciendo agg')    
        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby('key_value').agg(agg_rcc)
        rcc_test_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

rcc_test_ = pd.concat(list_rcc_test_agg, axis=1)
del rcc_test, list_rcc_test_agg

haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo cod_instit_financiera
hacie

In [9]:
# keep_columns = list(set(rcc_train_.columns).intersection(rcc_test_.columns))
# rcc_train_ = rcc_train_[keep_columns].copy()
# rcc_test_ = rcc_test_[keep_columns].copy()
train = rcc_train_.copy()
test = rcc_test_.copy()
del rcc_train_, rcc_test_

In [10]:
### imputar datos categoricos
var_categ = [i for i in train.columns if 'mode' in i]
for i in var_categ:
    train[i] = train[i].fillna(train[i].value_counts().index[0])
    test[i] = test[i].fillna(train[i].value_counts().index[0])

In [11]:
### mean encoding
### ojo, en el test hay missing values pero pocos
train = pd.concat([train, y_train], axis=1)
for i in var_categ:
    me = train.groupby([i])['target'].mean().to_dict() 
    train[f'{i}_me'] =  train[i].map(me) 
    test[f'{i}_me'] =  test[i].map(me)
    ### imputar datos faltantes
    test[f'{i}_me'] =  test[f'{i}_me'].fillna(train[f'{i}_me'].median())
    
train.drop('target', axis=1, inplace=True)
train.drop(var_categ, axis=1, inplace=True)
test.drop(var_categ, axis=1, inplace=True)

## Load SE

In [12]:
se_train = pd.read_csv('/kaggle/input/interbank20/se_train.csv', index_col='key_value')
se_test = pd.read_csv('/kaggle/input/interbank20/se_test.csv', index_col='key_value')
# edad y cod_ubi tienen valores nulos con cerca de 1% cada uno

In [13]:
dict_ = {'sexo':'int32',
 'est_cvl':'int32',
 'sit_lab':'int32',
 'cod_ocu':'int32',
 'ctd_hijos':'int32',
 'flg_sin_email':'int32',
 'ctd_veh':'int32',
 'lgr_vot':'int32',
 'prv':'int32',
 'dto':'int32',
 'rgn':'int32',
 'tip_lvledu':'int32'}
se_train = se_train.astype(dict_)
se_test = se_test.astype(dict_)

In [14]:
### imputar datos faltantes en la base se_[train/test]
se_train['edad'].fillna(se_train['edad'].median(), inplace=True)
se_test['edad'].fillna(se_train['edad'].median(), inplace=True)
se_train['cod_ubi'].fillna(se_train['cod_ubi'].median(), inplace=True)
se_test['cod_ubi'].fillna(se_train['cod_ubi'].median(), inplace=True)

In [15]:
var_categ=['sit_lab','lgr_vot','sexo','rgn','tip_lvledu','flg_sin_email','prv','est_cvl','cod_ocu','dto']
# var_caunti=['edad','ctd_hijos','ctd_veh','cod_ubi']
se_train['target'] = y_train.loc[se_train.index]
for i in var_categ:
    me = se_train.groupby([i])['target'].mean().to_dict() 
    se_train[f'{i}_me'] =  se_train[i].map(me) 
    se_test[f'{i}_me'] =  se_test[i].map(me)
    ### imputar datos faltantes
    se_test[f'{i}_me'] =  se_test[f'{i}_me'].fillna(se_train[f'{i}_me'].median())
se_train.drop('target', axis=1, inplace=True)
se_train.drop(var_categ, axis=1, inplace=True)
se_test.drop(var_categ, axis=1, inplace=True)

In [16]:
train = train.join(se_train) ### Ojo 57 valores perdidos
test = test.join(se_test)
del se_train, se_test
# train.drop(list(se_train.columns), axis=1, inplace=True)
# test.drop(list(se_train.columns), axis=1, inplace=True)

In [17]:
### imputar valores faltantes
var_caunti = ['lgr_vot_me','sit_lab_me','cod_ubi','tip_lvledu_me','prv_me','rgn_me',
 'ctd_hijos','edad','cod_ocu_me','ctd_veh','est_cvl_me','dto_me','flg_sin_email_me','sexo_me']
for i in var_caunti:
    train[i] = train[i].fillna(train[i].median())
    test[i] = test[i].fillna(train[i].median())
gc.collect()

44

## Load Censo

In [18]:
# censo_train = pd.read_csv('/kaggle/input/interbank20/censo_train.csv', index_col='key_value')
# censo_test = pd.read_csv('/kaggle/input/interbank20/censo_test.csv', index_col='key_value')

In [19]:
# var_cuanti = ['lon_x', 'lat_y','score','pob','pob_a','pob_b','pob_c','pob_d','pob_e','dehogar2','dehogar3',
#               'dehogar4', 'dehogar5', 'dehogar6', 'dehogar7', 'dehogar8','tipviv1', 'tipviv3', 'tipviv4',
#               'tipviv6', 'tipviv7', 'tipviv9', 'matpar2', 'matpar3', 'matpar4', 'matpar5', 'matpar6', 'matpar7', 'matpar8',
#               'abaagu2', 'abaagu3', 'abaagu4', 'abaagu5', 'abaagu6', 'abaagu7', 'abaagu8', 'abaagu9',
#               'vivten1', 'vivten2', 'vivten4', 'vivten5', 
#               'mattec2', 'mattec3', 'mattec4', 'mattec5', 'mattec6', 'mattec7', 'mattec8',
#               'agudia2', 'agundia1', 'agundia2', 'agundia3', 'agundia4', 'agundia5', 'agundia6', 'pagaguno',
#               'pagagu2', 'pagagu3', 'pagagu5', 'hig3', 'hig4', 'hig5', 'hig6', 'hig7', 'hig8', 'alupuno',
#               'cocteno', 'refteno', 'celteno', 'motten', 'lanten', 'adumay', 'pafisis', 'pafue', 'paotr', 'lee_no', 
#               'estud_1', 'estud_5', 'estud_6', 'profe_1', 'profe_5', 'profe_6', 'profe_7', 'profe_8', 'profe_9','profe_10',
#               'acteco2', 'acteco3', 'acteco4', 'acteco5', 'acteco6', 'acteco9', 'acteco12', 'acteco14', 'acteco15', 'acteco18',
#               'acteco19', 'acteco20', 'acteco21','ocup_1', 'ocup_4', 'ocup_5']
# var_categ = ['zon_sis', 'zon_der', 'zon_inu','pred','flag_censo']
# censo_train = censo_train[var_cuanti+var_categ].copy()
# censo_test = censo_test[var_cuanti+var_categ].copy()

In [20]:
# train = train.join(censo_train)
# test = test.join(censo_test)
# del censo_train, censo_test

In [21]:
# for i in var_cuanti:
#     train[i] = train[i].fillna(train[i].median())
#     test[i] = test[i].fillna(train[i].median())
# for i in var_categ:
#     train[i] = train[i].fillna(train[i].value_counts().index[0])
#     test[i] = test[i].fillna(train[i].value_counts().index[0])

## Load Sunat

In [22]:
sunat_train = pd.read_csv('/kaggle/input/interbank20/sunat_train.csv')
sunat_test = pd.read_csv('/kaggle/input/interbank20/sunat_test.csv')

In [23]:
dict_ = dict(zip(list(sunat_train.dtypes[sunat_train.dtypes == int].index), ['int32']*len(sunat_train.dtypes[sunat_train.dtypes == int].index)))
sunat_train = sunat_train.astype(dict_)
sunat_test = sunat_test.astype(dict_)

In [24]:
import statistics as stat
import numpy as np
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=np.nan
    return moda
from tqdm.notebook import tqdm 
moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_sunat = {'tipcontribuyente':['nunique',moda],
           'tippersona':['nunique',moda],
           'ciiu':['nunique', moda],
           'ubigeo':['nunique',moda],
           'condiciondomicilio':['nunique',moda],
           'estadocontribuyente':['nunique',moda],
           'codvia':['nunique',moda],
           'codzona':['nunique',moda],
           'contabilidad':['nunique',moda],
           'facturacion':['nunique',moda],
           'domiciliado':['nunique',moda],
           'comercioexterior':['nunique',moda],
           'cargorele':['nunique',moda],
           'codentidadtributo':['nunique',moda],
           'estadotributo':['nunique',moda],
           'fecalta':['sum', 'nunique'],
           'fecbaja':['sum', 'nunique'],
          }

In [25]:
sunat_train_ = sunat_train.groupby('key_value').agg(agg_sunat)
sunat_train_.columns = [i+'_'+j for i,j in sunat_train_.columns]
sunat_test_ = sunat_test.groupby('key_value').agg(agg_sunat)
sunat_test_.columns = [i+'_'+j for i,j in sunat_test_.columns]
del sunat_train, sunat_test

In [26]:
train = train.join(sunat_train_)
test = test.join(sunat_test_)
train.shape, test.shape

((358487, 391), (396666, 2253))

In [64]:
# var_aux = list((test.isna().sum().sort_values()/test.shape[0]).tail(34).index)
# var_categ = [i for i in var_aux if 'mode' in i]
# var_cuanti = [i for i in var_aux if 'mode'not in i]
# len(var_categ), len(var_cuanti)

(15, 19)

In [65]:
# for i in var_cuanti:
#     train[i] = train[i].fillna(train[i].median())
#     test[i] = test[i].fillna(train[i].median())
# for i in var_categ:
#     train[i] = train[i].fillna(train[i].value_counts().index[0])
#     test[i] = test[i].fillna(train[i].value_counts().index[0])

## Training

In [66]:
keep_columns = set(train.columns).intersection(test.columns)
train=train[keep_columns].copy()
test=test[keep_columns].copy()
train.shape, test.shape

((358487, 391), (396666, 391))

In [73]:
# !pip install catboost
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import numpy as np
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = CatBoostClassifier(n_estimators=1000,  eval_metric = 'AUC', max_depth = 6)
    learner.fit(Xt, yt,  early_stopping_rounds=10, 
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50) # cat_features = var_categ_index,
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
    gc.collect()
test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))]))

********** 0 **********
Learning rate set to 0.128212
0:	test: 0.7178506	test1: 0.7218432	best: 0.7218432 (0)	total: 323ms	remaining: 5m 23s
50:	test: 0.8246350	test1: 0.8223893	best: 0.8223893 (50)	total: 22.5s	remaining: 6m 59s
100:	test: 0.8339113	test1: 0.8300676	best: 0.8300676 (100)	total: 43.8s	remaining: 6m 30s
150:	test: 0.8400729	test1: 0.8340192	best: 0.8340192 (150)	total: 1m 4s	remaining: 6m 3s
200:	test: 0.8445619	test1: 0.8366574	best: 0.8366574 (200)	total: 1m 24s	remaining: 5m 34s
250:	test: 0.8477564	test1: 0.8378113	best: 0.8378113 (250)	total: 1m 43s	remaining: 5m 10s
300:	test: 0.8512515	test1: 0.8388767	best: 0.8388767 (300)	total: 2m 3s	remaining: 4m 47s
350:	test: 0.8539742	test1: 0.8396364	best: 0.8396706 (348)	total: 2m 23s	remaining: 4m 24s
400:	test: 0.8567066	test1: 0.8402545	best: 0.8402685 (396)	total: 2m 43s	remaining: 4m 4s
450:	test: 0.8592941	test1: 0.8408570	best: 0.8408570 (450)	total: 3m 3s	remaining: 3m 43s
500:	test: 0.8616588	test1: 0.8413375	be

In [ ]:

# roc auc estimado:  0.860778375567941  # roc auc estimado:  0.8287296558597061 benchmark

# roc auc estimado:  0.8235666881996343 solo considerando rcc, inputando datos categoricos. con 4 mes
# roc auc varianza:  0.0005453571957666847

# roc auc estimado:  0.8236751116856392 solo considerando rcc, inputando datos categoricos. con 6 mes
# roc auc varianza:  0.0007266775450365375

# roc auc estimado:  0.8240426706542976 solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0006480438484228306

# roc auc estimado:  0.8358995025496325   solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.000799546878111712  y incluyendo se sin imputar datos

# roc auc estimado:  0.8362318848230975    solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0006670589830583723 y incluyendo la base se imputando edad y cod_ubi

# roc auc estimado:  0.8357779560666587  solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0008281941487650447 y incluyendo la base se imputando edad y cod_ubi. y en base final 

# roc auc estimado:  0.8386997825126665  solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0007378701891037694 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding

# roc auc estimado:  0.8390378213410135 solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0007879483415566218 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding. Impuntando la base final
#### 0.83049 es el score en el leaderboard

# roc auc estimado:  0.860778375567941 solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0007522563589463705 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding. 
#                                         Add censo sin imputar datos
#### Hay un sobreajuste de los datos (0.81290)

# roc auc estimado:  0.8601038804961594 solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0007408029176933889 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding. 
#                                         Add censo. Inputando datos en la base final 
####  Hay un sobreajuste de los datos (0.81335)

# roc auc estimado:  0.8403745150148406 solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0007145134831505686 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding.
#                                         Add sunat, sin imputar datos 
#### 0.83169 es el score en el leaderboard

# roc auc estimado:  0.8404409620195707 solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0007187836690898979 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding.
#                                         Add sunat, imputando datos en la base final

In [41]:
fi.sort_values(ascending=False).head(50)
# COD_CLASIFICACION_DEUDOR_max_ult1mes              0.054579
# COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.046220
# tipo_credito_11_saldo_ult12meses                  0.043322
# tipo_credito_11_saldo_ult1meses                   0.040848
# tipo_credito_12_saldo_ult1meses                   0.040650
# RIESGO_DIRECTO_-1_saldo_ult12meses                0.039103
# COD_CLASIFICACION_DEUDOR_0_saldo_ult1meses        0.037563
# RIESGO_DIRECTO_1_saldo_ult1meses                  0.036770

# COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.058760
# tipo_credito_11_saldo_ult12meses                  0.039706
# tipo_credito_11_saldo_ult1meses                   0.038419
# RIESGO_DIRECTO_1_saldo_ult1meses                  0.035330
# tipo_credito_12_saldo_ult1meses                   0.032713
# RIESGO_DIRECTO_-1_saldo_ult12meses                0.031817
# cod_instit_financiera_mode_ult1mes                0.029203
# cod_instit_financiera_mode_ult12mes               0.026591

# tipo_credito_11_saldo_ult12meses                  0.041308
# COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.040848
# COD_CLASIFICACION_DEUDOR_max_ult1mes              0.038907
# RIESGO_DIRECTO_1_saldo_ult1meses                  0.035023
# cod_instit_financiera_mode_ult1mes_me             0.034799
# RIESGO_DIRECTO_-1_saldo_ult12meses                0.032097
# cod_instit_financiera_mode_ult12mes_me            0.028594
# tipo_credito_12_saldo_ult1meses                   0.028358

# dto_me                                            0.060329
# COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.034688
# cod_instit_financiera_mode_ult1mes_me             0.033413
# tip_lvledu_me                                     0.031962
# tipo_credito_12_saldo_ult1meses                   0.029074
# RIESGO_DIRECTO_1_saldo_ult1meses                  0.028161
# edad                                              0.024312
# tipo_credito_11_saldo_ult12meses                  0.024109

# dto_me                                            0.064928
# tip_lvledu_me                                     0.030977
# COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.029493
# RIESGO_DIRECTO_1_saldo_ult1meses                  0.028540
# COD_CLASIFICACION_DEUDOR_sum_ult2mes              0.028324
# cod_instit_financiera_mode_ult1mes_me             0.027232
# tipo_credito_12_saldo_ult1meses                   0.027068
# COD_CLASIFICACION_DEUDOR_max_ult1mes              0.025515

# dto_me                                            0.055801
# COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.044139
# COD_CLASIFICACION_DEUDOR_max_ult1mes              0.036097
# cod_instit_financiera_mode_ult1mes_me             0.026466
# abaagu5                                           0.024264
# tipo_credito_12_saldo_ult1meses                   0.021925
# tipo_credito_11_saldo_ult1meses                   0.021864
# cod_instit_financiera_nunique_ult1mes             0.021075

# dto_me                                            0.053591
# COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.038489
# COD_CLASIFICACION_DEUDOR_max_ult1mes              0.026788
# cod_instit_financiera_mode_ult1mes_me             0.024168
# COD_CLASIFICACION_DEUDOR_0_saldo_ult1meses        0.023845
# zon_sis                                           0.021759
# tipo_credito_12_saldo_ult1meses                   0.021530
# RIESGO_DIRECTO_1_saldo_ult1meses                  0.020410

# dto_me                                            0.060515
# COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.037680
# cod_instit_financiera_mode_ult1mes_me             0.030271
# tipo_credito_12_saldo_ult1meses                   0.029791
# tip_lvledu_me                                     0.029224
# RIESGO_DIRECTO_1_saldo_ult1meses                  0.024276
# edad                                              0.024108
# tipo_credito_11_saldo_ult12meses                  0.024044

dto_me                                            0.060515
COD_CLASIFICACION_DEUDOR_sum_ult1mes              0.037680
cod_instit_financiera_mode_ult1mes_me             0.030271
tipo_credito_12_saldo_ult1meses                   0.029791
tip_lvledu_me                                     0.029224
RIESGO_DIRECTO_1_saldo_ult1meses                  0.024276
edad                                              0.024108
tipo_credito_11_saldo_ult12meses                  0.024044
COD_CLASIFICACION_DEUDOR_0_saldo_ult1meses        0.023251
RIESGO_DIRECTO_-1_saldo_ult12meses                0.022323
tipo_credito_11_saldo_ult1meses                   0.021568
cod_instit_financiera_mode_ult12mes_me            0.021162
COD_CLASIFICACION_DEUDOR_max_ult1mes              0.018876
COD_CLASIFICACION_DEUDOR_sum_ult2mes              0.016830
RIESGO_DIRECTO_-1_saldo_ult1meses                 0.015959
cod_instit_financiera_mode_ult6mes_me             0.015362
cod_instit_financiera_nunique_ult1mes             0.0153

In [42]:
learner_ = CatBoostClassifier(n_estimators=1000, eval_metric = 'AUC', max_depth = 6)
learner_.fit(train, y_train,  early_stopping_rounds=10, verbose = 50)

Learning rate set to 0.126968
0:	total: 394ms	remaining: 6m 33s
50:	total: 26s	remaining: 8m 3s
100:	total: 49.2s	remaining: 7m 17s
150:	total: 1m 12s	remaining: 6m 48s
200:	total: 1m 35s	remaining: 6m 20s
250:	total: 1m 56s	remaining: 5m 47s
300:	total: 2m 18s	remaining: 5m 22s
350:	total: 2m 41s	remaining: 4m 58s
400:	total: 3m 3s	remaining: 4m 34s
450:	total: 3m 26s	remaining: 4m 11s
500:	total: 3m 48s	remaining: 3m 47s
550:	total: 4m 10s	remaining: 3m 23s
600:	total: 4m 32s	remaining: 3m
650:	total: 4m 55s	remaining: 2m 38s
700:	total: 5m 16s	remaining: 2m 15s
750:	total: 5m 37s	remaining: 1m 51s
800:	total: 5m 59s	remaining: 1m 29s
850:	total: 6m 21s	remaining: 1m 6s
900:	total: 6m 43s	remaining: 44.3s
950:	total: 7m 5s	remaining: 21.9s
999:	total: 7m 27s	remaining: 0us


In [43]:
fi_ = pd.Series(learner_.feature_importances_ / learner_.feature_importances_.sum(), index=train.columns)
fi_.sort_values(ascending=False).head(50)

dto_me                                         0.064845
tip_lvledu_me                                  0.025896
edad                                           0.025402
cod_instit_financiera_mode_ult1mes_me          0.024748
RIESGO_DIRECTO_1_saldo_ult1meses               0.024481
tipo_credito_12_saldo_ult1meses                0.023930
COD_CLASIFICACION_DEUDOR_sum_ult1mes           0.023239
tipo_credito_11_saldo_ult1meses                0.022092
cod_instit_financiera_mode_ult12mes_me         0.021052
COD_CLASIFICACION_DEUDOR_max_ult1mes           0.020561
tipo_credito_11_saldo_ult12meses               0.020170
COD_CLASIFICACION_DEUDOR_0_saldo_ult1meses     0.019520
RIESGO_DIRECTO_1_saldo_ult12meses              0.017500
RIESGO_DIRECTO_-1_saldo_ult12meses             0.016519
RIESGO_DIRECTO_-1_saldo_ult2meses              0.016001
COD_CLASIFICACION_DEUDOR_0_saldo_ult12meses    0.014630
saldo_max_ult12mes                             0.014566
cod_instit_financiera_mode_ult2mes_me          0

In [44]:
test_probs_ = pd.Series(learner_.predict_proba(test)[:, -1], index=test.index, name="target")
test_probs_.shape

(396666,)

In [45]:
test_probs_.name = "target"
test_probs_.to_csv("test17.csv")

<a href='test17.csv'>Download csv</a>